In [1]:
import json
from pathlib import Path

import cv2
import numpy as np
import tifffile
from tqdm import tqdm

In [2]:
data_dir = Path("./data")

In [ ]:
# Generate list of training image filenames
train_file_names = [f"train_{i}.tif" for i in range(176)]  
class_names = ["grassland_shrubland", "logging", "mining", "plantation"]

# Load raw annotation JSON
with open(data_dir / "train_annotations.json", "r") as f:
    raw_annotations = json.load(f)

# Map each filename to its class-wise polygon annotations
annotations: dict[str, dict[str, list[list[float]]]] = {}
for fn in tqdm(train_file_names):
    ann = {class_name: [] for class_name in class_names}

    for img in raw_annotations["images"]:
        if img["file_name"] == fn:
            for a in img["annotations"]:
                ann[a["class"]].append(a["segmentation"])

    annotations[fn] = ann


100%|██████████| 176/176 [00:00<00:00, 57975.14it/s]


In [ ]:
# Create directory to save masks
mask_save_dir = data_dir / "train_masks"
mask_save_dir.mkdir(parents=True, exist_ok=True)

# Generate and save multi-channel masks
for fn in tqdm(train_file_names):
    mask = np.zeros((4, 1024, 1024), dtype=np.uint8)
    anns = annotations[fn]
    for class_idx, class_name in enumerate(class_names):
        polygons = anns[class_name]
        for poly in polygons:
            pts = np.array(poly, dtype=np.int32).reshape(-1, 2)
            cv2.fillPoly(mask[class_idx], [pts], 255)

    np.save(mask_save_dir / fn.replace(".tif", ".npy"), mask)


100%|██████████| 176/176 [00:01<00:00, 167.80it/s]


In [ ]:
# Save RGB image and class masks as a side-by-side PNG visualization
vis_save_dir = data_dir / "vis_train"
vis_save_dir.mkdir(parents=True, exist_ok=True)

for fn in tqdm(train_file_names):
    mask = np.load(mask_save_dir / fn.replace(".tif", ".npy"))  # (4, 1024, 1024)
    vis_masks = [np.zeros((1024, 1024, 3), dtype=np.uint8) for _ in range(4)]

    for class_idx, class_name in enumerate(class_names):
        vis_masks[class_idx][mask[class_idx] > 0] = [255, 0, 0]
        cv2.putText(vis_masks[class_idx], class_name, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    vis_image = tifffile.imread(data_dir / "train_images" / fn)
    vis_image = vis_image[:, :, [1, 2, 3]]  # Select bands B2, B3, B4
    vis_image = np.nan_to_num(vis_image, nan=0)
    vis_image = (vis_image / 8).clip(0, 255).astype(np.uint8)

    partition = np.ones((1024, 5, 3), dtype=np.uint8) * 255  # white spacer
    vis = np.concatenate([vis_image] + [partition, vis_masks[0], partition, vis_masks[1], partition, vis_masks[2], partition, vis_masks[3]], axis=1)

    cv2.imwrite(vis_save_dir / fn.replace(".tif", ".png"), vis)


100%|██████████| 176/176 [00:56<00:00,  3.09it/s]
